In [770]:
import sys
sys.path.append("C:/Users/jonor/Documents/GitHub/nanonis/")
import nanonis
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
%matplotlib qt
spectra=nanonis.biasSpectroscopy()

# Useful functions

In [771]:
def fermi(x,T):
    return np.divide(1,1+np.exp(x/T))


def BCS(x,delta,eta):
    return np.sign(x)*np.imag(np.divide(np.abs(x+1j*eta),np.sqrt(delta**2-(x+1j*eta)**2)))

def coulomb(x,Ec,Q0):
    return np.heaviside(x-Ec-Q0,0)+np.heaviside(-x-Ec+Q0,0)

def conv_dynes(x,delta1,delta2,eta,T,Ec,Q0):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    a,b=np.meshgrid(y,y)
    cou=np.gradient(-np.dot(BCS(a+b,delta1,eta)*fermi(a+b,T),coulomb(y,Ec,Q0))+np.dot(BCS(a+b,delta1,eta),fermi(y,T)*coulomb(y,Ec,Q0)))
    a,b=np.meshgrid(y,x)
    z=np.gradient(-np.dot(BCS(a+b,delta2,eta)*fermi(a+b,T),cou)+np.dot(BCS(a+b,delta2,eta),cou*fermi(y,T)))
    return z/z[0]

def temp(x,delta1,delta2,eta,T):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    a,b=np.meshgrid(y,x)
    return -np.dot(BCS(a+b,delta2,eta)*fermi(a+b,T),BCS(y,delta1,eta))+np.dot(BCS(a+b,delta2,eta),BCS(y,delta1,eta)*fermi(y,T))

def temp2(V,delta1,delta2,eta,T):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    return np.sum(-BCS(y+V,delta2,eta)*fermi(y+V,T)*BCS(y,delta1,eta)+BCS(y+V,delta2,eta)*BCS(y,delta1,eta)*fermi(y,T))

def simple_fit_func(V,delta1,delta2,Ec,T):
    delta=np.max(delta1,delta2)
    x=np.linspace(-10*delta,10*delta)
    rho1=BCS()

def gap(x,delta1,Bc1,Ec,a):
    d1=(0.0013*np.real(np.sqrt(1-((x+0.0000*1j)/800)**2)))*np.heaviside(-x+800,0)
    d2=delta1*np.real(np.sqrt(1-((x+0.0000*1j)/Bc1)**2))*np.heaviside(-x+Bc1,0)
    d3=(-a*x)*np.heaviside(x-Bc1,0)
    return Ec+d1+d2+d3
   
def coulomb_T(x,Ec,T):
    y=np.linspace(-3*np.min(x),3*np.min(x),2000)
    a,b=np.meshgrid(y,x)
    return np.gradient(-np.dot(fermi(a+b,T),coulomb(y,Ec,Q0))+fermi(y,T)*coulomb(y,Ec,Q0))


In [772]:
x=np.linspace(0,2000,1000)
y=gap(x,0.0013,350,0.001,0.000001)
plt.plot(x,y)

In [145]:
x=np.linspace(-4,4,1000)
plt.plot(x,conv_dynes(x,1,1,0.1,0.1,0,0))

# File management #

In [796]:
data=pd.read_excel("C:/Users/jonor/Documents/GitHub/nanonis/Order.xls")
path='C:/Users/jonor/Desktop/PhD/Lanak/Pb on Gr (analysis)/2023.07.26 - 1st evap/'


## B Field Files

In [797]:
Bfield_files=data.iloc[:,2]
Bfields=data.iloc[:,3]
size_dat=data.iloc[:,7]
height_dat=data.iloc[:,8]
temp=data.iloc[:,6]
temp_files=data.iloc[:,5]

# B Field 

In [779]:
## All files
dat=[1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,21,23,24]
BFields=[]
BFiles=[]
size=[]
height=[]
for i in dat:
    a=[int(j) for j in Bfields[i].split(',')]
    b=Bfield_files[i].split(',')    
    files=[path+i for i in b]
    BFields.append(a)
    BFiles.append(files)
    size.append(size_dat[i])
    height.append(height_dat[i])
size=np.array(size)
height=np.array(height)

In [734]:
i=12
a=[int(j) for j in Bfields[i].split(',')]
b=Bfield_files[i].split(',')
files=[path+i for i in b]


In [675]:
w_B=[1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,21,23,24]
files_B0=[]
size=[]
for j in w_B:
    a=[int(i) for i in Bfields[j].split(',')]
    b=Bfield_files[j].split(',')
    files=[path+i for i in b]
    field0=a.index(0)
    files_B0.append(files[field0])
    size.append(size_dat[j])


### Plots

In [722]:
y=[]
for i in files:
    spectra.load(i)
    y.append(spectra.conductance/spectra.conductance[0])
x=np.array(spectra.bias)
plt.imshow(y,aspect='auto',origin='lower',extent=(1000*x[0],1000*x[-1],a[-1],a[0]))
plt.ylabel('B(mT)')
plt.xlabel('Bias (meV)')

Text(0.5, 0, 'Bias (meV)')

### Some info

The simplest way to get the gao is to calculate it from the current. We take the points for which the current is zero.



In [782]:
h=0
Ec=[]
Eg=[]
for k in range(len(BFiles)):
    cur=[]
    for i in BFiles[k]:
        spectra.load(i)
        cur.append(spectra.current)

    t=[]
    for j in range(len(BFiles[k])):
        y=np.array(cur[j])
        x=np.array(spectra.bias)
        xx=[]
        for i in range(len(y)):
            if np.abs(y[i]/y[0])<0.04:
                xx.append(x[i])
        t.append(xx[0]-xx[-1])
    T800=BFields[k].index(800)
    T0=BFields[k].index(0)
    Ec.append(t[T0])


plt.plot(size*height,Ec,'bo')


### Solme info

The critical field for the tip is around B=800mT, while the critical field for the island should be around B=350mT. Althought the first statement is prety robust, the second is not. 

The following program fit the spectra with a simple convolution. The free parameters will be the coulomb gap (Ec) the shift (Q0) and the dynes parameter (eta). We will suppose that the gaps decrease following the BCS theory with the critical fields that we wrote in the previous line. This first program is not a realistic fir of the system, we will only use it to get an idea of the volution of the full gap of the measurement, i.e., E+Delta1+Delta2. We will not make assupmtions about any of this parameters from the results of this analysis.

In [ ]:
fit=sc.optimize.curve_fit
def delta(Delta,B,Bc):
    return Delta*np.sqrt(1-(B/Bc)**2)

def conv_dynes(x,delta,eta,Ec):
    T=delta/10
    Q0=-0.00025
    y=np.linspace(-10*delta,10*delta,2000)
    a,b=np.meshgrid(y,x)
    z=np.gradient(-np.dot(BCS(a+b,delta,eta)*fermi(a+b,T),coulomb(y,Ec,Q0))+np.dot(BCS(a+b,delta,eta),fermi(y,T)*coulomb(y,Ec,Q0)))
    return z/z[0]


cond=[]
for i in files:
    spectra.load(i)
    cond.append(spectra.conductance)

gap=[]
for j in range(0,10):
    y=np.array(cond[j])[110:290]/np.array(cond[j])[110]
    x=np.array(spectra.bias)[110:290]
    temp=fit(conv_dynes,x,y,p0=[0.0013,0.0001,0.0012])
    plt.plot(x,y+0.5*j)
    p=temp[0]
    z=conv_dynes(x,p[0],p[1],p[2])
    plt.plot(x,z+0.5*j)
    gap.append(p[0]+p[2])
    print(j)

# Temperature dependence

In [820]:
data=[8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
T=[]
files=[]

for j in data:
    a=temp[j].split(',')
    T.append([float(i) for i in a])
    b=temp_files[j].split(',')
    files.append([path+i for i in b])

In [829]:
f=files[0][0]
spectra.load(f)
plt.plot(spectra.bias,spectra.conductance/spectra.conductance[0])